In [1]:
import bz2
import json
import requests as rq
import time

In [2]:
import lzma

In [3]:
dump = "/10TBdrive/datashare/Reddit/comments/RC_2017-01.bz2"
# dump = "/10TBdrive/datashare/Reddit/submissions/RS_2017-12.xz"

In [4]:
openedDump = bz2.BZ2File(dump, "r")
# openedDump = lzma.open(dump)

In [5]:
count = 0
for line in openedDump:
    if count > 10:
        break
    print(repr(line))
    count+=1

b'{"edited":false,"id":"dbumnpz","parent_id":"t1_dbulzrw","distinguished":null,"created_utc":1483228800,"author_flair_text":null,"author_flair_css_class":"NYAN","controversiality":0,"subreddit_id":"t5_22i0","retrieved_on":1485679711,"link_id":"t3_5lc6zb","author":"captnkaposzta","score":2,"gilded":0,"stickied":false,"body":"Beileid? Kiwi Fernsehgarten Trinkspiele retten jede Party.\\n\\nGesundes Neues\\n","subreddit":"de"}\n'
b'{"author":"CampyJejuni","score":3,"stickied":false,"gilded":0,"body":"Wrong subreddit mate.","subreddit":"TwoXChromosomes","controversiality":0,"author_flair_css_class":null,"author_flair_text":null,"subreddit_id":"t5_2r2jt","retrieved_on":1485679711,"link_id":"t3_5lai4x","created_utc":1483228800,"distinguished":null,"parent_id":"t1_dbum9w2","edited":false,"id":"dbumnq0"}\n'
b'{"distinguished":null,"created_utc":1483228800,"edited":false,"id":"dbumnq1","parent_id":"t3_5lb9zs","author":"Luigimario280","score":7,"stickied":false,"body":"Karma!","gilded":0,"subredd

In [111]:
def getJson(url):
    session_headers = {
    'Host': 'www.reddit.com',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    #'Referer': 'http://localhost:9000/notebooks/Reddit/Notebooks/SuicideWatchSubreddit.ipynb'
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-GB,en-US;q=0.8,en;q=0.6',
    'Cookie': "loid=00000000000gslzvdi.2.1507591930964.Z0FBQUFBQlozQWI3M243MTdFMUgzNWtQNzNVbzg5Z3NXYmxpLUdRTWVXdEpWQzBfbkxBSVNNVVF3dE1zQlN5cVMzeGtxZXlmT0JQY2JvTld2NWlVeGlGeFpkNnBYM2ZIYlNNR2xMWGFFS3hieDZpWEJ5OVlyTng0MUFZZjNYdjQ0ZURFNTFlSmlVRGw; edgebucket=T8nkTF0z7WPnNzlE8B; __gads=ID=ab9c1b0fcb7eaefc:T=1507591936:S=ALNI_Mao2tvEoyJzEDEuxxhfZBlOQHXlsA; eu_cookie_v2=3; _ga=GA1.2.1088810883.1507591931; __utma=55650728.1088810883.1507591931.1507912304.1507912553.14; __utmz=55650728.1507912553.14.11.utmcsr=reddit|utmccn=(not%20set)|utmcmd=hot|utmcct=comments; aa=1; _recentclicks2=t3_7cfzsx%2Ct3_7cfw08%2Ct3_7cdofu%2Ct3_7caog1%2Ct3_7cdvud; _recent_srs=t5_2qpzs%2Ct5_2qh1q%2Ct5_2qh49%2Ct5_2rtff%2Ct5_36buk%2Ct5_2r7yd%2Ct5_2rks3%2Ct5_3j2jr%2Ct5_2sumw%2Ct5_2xp2o; pc=r3; session_tracker=tPgx9U9yPt99jYRDqC.0.1510585492994.Z0FBQUFBQmFDYlNWcWZHclI5WWxDWW9kQ0dURk9FRU9BZjFJTURWZElzaldRbkROaEJUOWE1czRQUWxPUzUwOFFwY3R3ckI5VkNDUlhJc01NVEtEeHVWd25CdUpXWEI1X012RkJaS3cyNXdNbk5vOE9DSXp5ZlhuTzF3a2pxT2dLZ3laMDhtQlhib3k; initref=localhost"
    }

    client = rq.session()
    resp = client.get(url , headers=session_headers)
    if resp.status_code == 200:
        json_data = resp.json()
        return json_data
    else:
        print("Failed to get page")
        return None
def getSubredditId(json):
    if type(json) == dict:
        sub_id = json['data']['children'][0]['data']['subreddit_id']
        return sub_id
    else: 
        print("Invalid Format")
        return None

def makeSubUrl(name):
    return "https://www.reddit.com/r/" + name + "/.json"

def getSubIds(RedditList):
    subIds = {}
    for name in RedditList:
        json = getJson(makeSubUrl(name))
        print (getSubredditId(json))
        subIds[name] = getSubredditId(json)
    return subIds

def filterDumpbySub(filePath , subID_dict, limit=None):
    import json
    import bz2
    
    openedDump = bz2.BZ2File(filePath, "r")
    count = 0
    returnData = dict()
    print("=============Parsing============")
    print (subID_dict)
    st = time.time()
    for line in openedDump:
        lineData = json.loads(line.strip())
        if (limit!=None and count > limit):
            return returnData
        if lineData['subreddit'] in subID_dict:
            if lineData['subreddit'] not in returnData:
                returnData[lineData['subreddit']] = []
            postId = lineData['link_id'].strip().split('_')[1]
            
            if postId not in returnData[lineData['subreddit']]:
                returnData[lineData['subreddit']].append(postId)
        if count%100000 == 0:
            end = time.time()
            print("Done parsing %d posts in %d seconds",(count,(end-st)))
            st = time.time()
        count+=1
    return returnData


def filterSubmissionsbySub(filePath , subID_dict, limit=None):
    import json
    import bz2
    
    openedDump = bz2.BZ2File(filePath, "r")
    count = 0
    returnData = dict()
    print("=============Parsing============")
    print (subID_dict)
    st = time.time()
    for line in openedDump:
        lineData = json.loads(line.strip())
        if (limit!=None and count > limit):
            return returnData
        if lineData['subreddit_id'] in subID_dict:
            if lineData['subreddit_id'] not in returnData:
                returnData[lineData['subreddit_id']] = []
            postId = lineData['id']
            
            if postId not in returnData[lineData['subreddit_id']]:
                returnData[lineData['subreddit_id']].append(postId)
        if count%100000 == 0:
            end = time.time()
            print("Done parsing %d posts in %d seconds",(count,(end-st)))
            st = time.time()
        count+=1
    return returnData



def getSubmissionPermalinks(filePath , comments_Thresh = 20 , limit=None):
    import json
    import bz2
    import lzma
    
    ext = filePath.split('.')[-1]
    
    if ext == 'bz2':
        openedDump = bz2.BZ2File(filePath, "r")
    elif ext == 'xz':
        openedDump = lzma.open(dump)
    else:
        print ("Invalid file type")
    count = 0
    returnData = {}
    print("=============Parsing============")
    print (subID_dict)
    st = time.time()
    for line in openedDump:
        lineData = json.loads(line.strip())
        if lineData['num_comments'] > comments_Thresh:
            if (limit!=None and count > limit):
                return returnData
            try:
                returnData[lineData['id']] = lineData['permalink']
            except:
                print("Permalink not found, moving on!")
        if count%100000 == 0:
            end = time.time()
            print("Done parsing %d posts in %d seconds",(count,(end-st)))
            st = time.time()
        count+=1
    return returnData

In [112]:
subreddits = ["bipolar" , "BipolarReddit" , "BipolarSOs" , "Health" , "addiction", "depression","SuicideWatch","Anxiety","Chicago"] 

In [113]:
subID_dict = getSubIds(subreddits)

t5_2qnv4
t5_2qzru
t5_2zww6
t5_2qh9z
t5_2qwvc
t5_2qqqf
t5_2qpzs
t5_2qmij
t5_2qh2t


In [ ]:
subID_dict

{'Anxiety': 't5_2qmij',
 'BipolarReddit': 't5_2qzru',
 'BipolarSOs': 't5_2zww6',
 'Chicago': 't5_2qh2t',
 'Health': 't5_2qh9z',
 'SuicideWatch': 't5_2qpzs',
 'addiction': 't5_2qwvc',
 'bipolar': 't5_2qnv4',
 'depression': 't5_2qqqf'}

In [ ]:
# postData = filterDumpbySub(dump,subID_dict)
postData = getSubmissionPermalinks(dump,comments_Thresh= 10)

=============Parsing============
{'bipolar': 't5_2qnv4', 'BipolarReddit': 't5_2qzru', 'BipolarSOs': 't5_2zww6', 'Health': 't5_2qh9z', 'addiction': 't5_2qwvc', 'depression': 't5_2qqqf', 'SuicideWatch': 't5_2qpzs', 'Anxiety': 't5_2qmij', 'Chicago': 't5_2qh2t'}
Done parsing %d posts in %d seconds (0, 0.033522844314575195)
Done parsing %d posts in %d seconds (100000, 9.98129653930664)
Done parsing %d posts in %d seconds (200000, 10.41993522644043)
Done parsing %d posts in %d seconds (300000, 10.165717124938965)
Done parsing %d posts in %d seconds (400000, 10.193071365356445)
Done parsing %d posts in %d seconds (500000, 9.758701086044312)
Done parsing %d posts in %d seconds (600000, 9.903562545776367)
Done parsing %d posts in %d seconds (700000, 9.867722511291504)
Done parsing %d posts in %d seconds (800000, 10.346718549728394)
Done parsing %d posts in %d seconds (900000, 9.891074657440186)
Done parsing %d posts in %d seconds (1000000, 10.027259826660156)
Done parsing %d posts in %d seconds

In [ ]:
len(postData)

In [ ]:
import pickle as pkl
pkl.dump(file=open('/10TBdrive/datashare/Reddit/Submissions032017_atleast10.pkl','wb'),obj=postData)

In [97]:
postData[list(postData.keys())[1]]

'/r/nba/comments/5lcgjd/game_thread_new_york_knicks_1616_houston_rockets/'

In [31]:
# import pickle as pkl
# with open("/datasets_1/sagarj/IoPPN_collab/MentalHealth_2017_posts.pkl",'w') as f:
#     pkl.dump(postData,f)

In [32]:
len(list(set(postData['depression'])))

KeyError: 'depression'

In [ ]:
# import pickle as pkl
# with open("ChangemyView2017.pkl",'w') as f:
#     pkl.dump(postData['changemyview'],f)

In [ ]:
# with open("Filtered_posts_2017.json",'w') as f:
#     json.dump(postData,f)